In [1]:
import os
import numpy as np
import pandas as pd
from random import randint,seed
from tqdm import tqdm
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,MinMaxScaler 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.ensemble import GradientBoostingRegressor as GBT

In [2]:
"""
df1 = pd.read_csv('data_conv/new_dataset/conv1.csv')
df1['freq'] = 1200
print (df1.shape)
df2 = pd.read_csv('data_conv/new_dataset/conv2.csv')
df2['freq'] = 800
print (df2.shape)
df3 = pd.read_csv('data_conv/new_dataset/conv3.csv')
df3['freq'] = 600
print (df3.shape)
df4 = pd.read_csv('data_conv/new_dataset/conv4.csv')
df4['freq'] = 1000
print (df4.shape)
"""
df1 = pd.read_csv('data_conv/conv1.csv')
df1['freq'] = 1200
print (df1.shape)
df2 = pd.read_csv('data_conv/conv2.csv')
df2['freq'] = 800
print (df2.shape)
df3 = pd.read_csv('data_conv/conv3.csv')
df3['freq'] = 600
print (df3.shape)
df4 = pd.read_csv('data_conv/conv4.csv')
df4['freq'] = 1000
print (df4.shape)

(475000, 9)
(220400, 9)
(159400, 9)
(399600, 9)


In [3]:
df = pd.concat([df1, df2,df3,df4], ignore_index=True)

In [4]:
df.head()

,Unnamed: 0,input_size_h,input_size_w,channels,filter_count,filter_size,stride,runtime,freq
0,0,562,20,900,85,4,2,3.582578,1200
1,1,33,53,8,3,2,1,0.002315,1200
2,2,33,53,13,20,10,1,0.075039,1200
3,3,13,33,5,85,8,1,0.008288,1200
4,4,13,53,5,85,10,1,0.014262,1200


In [5]:
df = df.drop(columns=['Unnamed: 0'])
df.head()

,input_size_h,input_size_w,channels,filter_count,filter_size,stride,runtime,freq
0,562,20,900,85,4,2,3.582578,1200
1,33,53,8,3,2,1,0.002315,1200
2,33,53,13,20,10,1,0.075039,1200
3,13,33,5,85,8,1,0.008288,1200
4,13,53,5,85,10,1,0.014262,1200


In [6]:
df_ = df.dropna(axis=0)
df_.head()

,input_size_h,input_size_w,channels,filter_count,filter_size,stride,runtime,freq
0,562,20,900,85,4,2,3.582578,1200
1,33,53,8,3,2,1,0.002315,1200
2,33,53,13,20,10,1,0.075039,1200
3,13,33,5,85,8,1,0.008288,1200
4,13,53,5,85,10,1,0.014262,1200


In [7]:
df_['runtime_log'] = np.log(df_.runtime)
#df_['runtime_log'] = df_.runtime

/home/samavatian.1/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.
/home/samavatian.1/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
df_ = df_.dropna(axis=0)

In [9]:
data = df_[['freq','input_size_h','input_size_w' ,'channels','filter_count' ,'filter_size' ,'stride']]
data = data
data.head()

,freq,input_size_h,input_size_w,channels,filter_count,filter_size,stride
0,1200,562,20,900,85,4,2
1,1200,33,53,8,3,2,1
2,1200,33,53,13,20,10,1
3,1200,13,33,5,85,8,1
4,1200,13,53,5,85,10,1


In [10]:
#df_[(df_.input_size_h <=20 ) & (df_.input_size_w <= 20)& (df_.channels > 200)]# &(df_.channels < df_.filter_count)&(df_.stride==1)]


In [11]:
target = df_.runtime_log
target.head()

0    1.276083
1   -6.068346
2   -2.589747
3   -4.792947
4   -4.250157
Name: runtime_log, dtype: float64

In [18]:
scaler = StandardScaler() 
scaler.fit(data)  
data_ = scaler.transform(data)
#data_ = (data-data.mean())/data.std()

In [19]:
np.asarray(data.mean())

array([949.78091873, 141.61134276, 141.62936396, 269.15586572,
       267.54293286,   5.36742049,   1.50212014])

In [20]:
np.asarray(data.std())

array([221.45979591, 169.59065736, 170.46716941, 406.7464412 ,
       403.56878583,   2.82131915,   0.50000434])

In [21]:
X_train, X_test, y_train, y_test = train_test_split(data_, target, test_size=0.2, random_state=0)

In [22]:
new_df = pd.DataFrame(X_train)
new_df.describe()

,0,1,2,3,4,5,6
count,22640.000000,22640.000000,22640.000000,22640.000000,22640.000000,22640.000000,22640.000000
mean,-0.000846,-0.001940,-0.000561,-0.003590,-0.004186,-0.005141,-0.000795
std,1.002787,0.997823,0.997765,0.998471,0.996595,0.998689,1.000025
min,-1.579461,-0.817343,-0.813246,-0.654365,-0.655520,-1.193584,-1.004249
25%,-0.676347,-0.717100,-0.713519,-0.629779,-0.630741,-1.193584,-1.004249
50%,0.226768,-0.522511,-0.519930,-0.531436,-0.531624,-0.484683,0.995769
75%,1.129882,0.456334,0.453882,0.201220,0.206801,0.933119,0.995769
max,1.129882,2.478887,2.466035,2.881070,2.907752,1.642020,0.995769


In [23]:
model = LinearRegression()
model.fit(X_train, y_train)
print('accuracy: ',model.score(X_test, y_test)*100)

('accuracy: ', 78.1128334796877)


In [24]:
model.coef_

array([-0.24460898,  1.21644393,  1.21837041,  1.46095085,  1.03541385,
        1.21496509, -0.62244654])

In [25]:
model.intercept_

-1.0133672833846008

In [26]:
l = [[1000,256,256,3,64,3,1],[1000,32,32,512,512,3,1],[1000,14,14,512,1024,3,1],[1000,50,50,78,423,4,1]]
std = np.asarray(data.std())
mean = np.asarray(data.mean())
l = np.asarray(l)
l = (l - mean)/std
l

array([[ 0.22676387,  0.67449858,  0.67092471, -0.65435327, -0.50435747,
        -0.83911829, -1.00423157],
       [ 0.22676387, -0.6463289 , -0.64311131,  0.59704059,  0.60573829,
        -0.83911829, -1.00423157],
       [ 0.22676387, -0.75246682, -0.74870349,  0.59704059,  1.87441917,
        -0.83911829, -1.00423157],
       [ 0.22676387, -0.54019098, -0.53751913, -0.46996322,  0.38520587,
        -0.48467416, -1.00423157]])

In [27]:
np.power(10,model.predict(l))/1000

array([4.97133596e-05, 2.92692366e-05, 3.32858706e-04, 2.33145705e-06])

In [28]:
model = MLPRegressor(hidden_layer_sizes=1024,learning_rate='invscaling',early_stopping=True,max_iter=10000)
model.fit(X_train, y_train)
print('accuracy: ',model.score(X_test, y_test)*100)
pred = model.predict(X_test)
print 'nonzero: ',pred.shape[0]-pred[pred<0].shape[0],'/',pred.shape[0]

('accuracy: ', 99.60658866049941)
nonzero:  2205 / 5660


In [29]:
bias = np.asarray(model.intercepts_)
arr = np.asarray(model.coefs_)

In [30]:
if os.path.exists("params.txt"):
    os.remove("params.txt")
f=open('params.txt','ab')
print >> f, "L0"+" W"+" "+str(arr[0].shape[0])+" "+str(arr[0].shape[1]) 
np.savetxt(f,arr[0])
print >> f, "L0"+" B"+" "+str(bias[0].shape[0])+" 1"
np.savetxt(f,bias[0])

print >> f, "L1"+" W"+" "+str(arr[1].shape[0])+" "+str(arr[1].shape[1]) 
np.savetxt(f,arr[1])
print >> f, "L1"+" B"+" "+str(bias[1].shape[0])+" 1"
np.savetxt(f,bias[1])
print >> f, "mean"
np.savetxt(f,np.asarray(data.mean()))
print >> f, "std"
np.savetxt(f,np.asarray(data.std()))
f.close()

In [67]:
print X_test.shape
np.savetxt('input.txt',X_test)

print model.predict(X_test).shape
np.savetxt('output.txt',model.predict(X_test))


(3653, 7)
(3653,)


In [68]:
test = pd.read_csv('model.csv')
test.head()

,Unnamed: 0,layer,#filters,size,stride,padding,input,c (input),output,c (output),time (sec)
0,0,conv,32,3.0,1.0,1.0,448.0,3.0,448.0,32.0,0.767809
1,1,max,NaN,2.0,2.0,0.0,448.0,32.0,224.0,32.0,0.249073
2,2,conv,64,3.0,1.0,1.0,224.0,32.0,224.0,64.0,1.464385
3,3,max,NaN,2.0,2.0,0.0,224.0,64.0,112.0,64.0,0.126436
4,4,conv,128,3.0,1.0,1.0,112.0,64.0,112.0,128.0,1.077104


In [69]:
test_conv = test[test.layer=='conv']
test_conv['freq'] = 1000
test_conv = test_conv.reset_index()
test_conv

/home/samavatian.1/.local/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,index,Unnamed: 0,layer,#filters,size,stride,padding,input,c (input),output,c (output),time (sec),freq
0,0,0,conv,32,3.0,1.0,1.0,448.0,3.0,448.0,32.0,0.767809,1000
1,2,2,conv,64,3.0,1.0,1.0,224.0,32.0,224.0,64.0,1.464385,1000
2,4,4,conv,128,3.0,1.0,1.0,112.0,64.0,112.0,128.0,1.077104,1000
3,5,5,conv,64,1.0,1.0,0.0,112.0,128.0,112.0,64.0,0.191137,1000
4,6,6,conv,128,3.0,1.0,1.0,112.0,64.0,112.0,128.0,1.068897,1000
5,8,8,conv,256,3.0,1.0,1.0,56.0,128.0,56.0,256.0,0.880570,1000
6,9,9,conv,128,1.0,1.0,0.0,56.0,256.0,56.0,128.0,0.132695,1000
7,10,10,conv,256,3.0,1.0,1.0,56.0,128.0,56.0,256.0,0.881454,1000
8,12,12,conv,512,3.0,1.0,1.0,28.0,256.0,28.0,512.0,0.817290,1000
9,13,13,conv,256,1.0,1.0,0.0,28.0,512.0,28.0,256.0,0.108569,1000


In [70]:
test_conv_input = test_conv[['freq','input','input' ,'c (input)','#filters' ,'size' ,'stride']]
test_conv_input.head()
#test_conv_target = np.log10(test_conv['time (sec)']*1000)                      

,freq,input,input,c (input),#filters,size,stride
0,1000,448.0,448.0,3.0,32,3.0,1.0
1,1000,224.0,224.0,32.0,64,3.0,1.0
2,1000,112.0,112.0,64.0,128,3.0,1.0
3,1000,112.0,112.0,128.0,64,1.0,1.0
4,1000,112.0,112.0,64.0,128,3.0,1.0


In [71]:
test_conv_input_ = scaler.transform(test_conv_input)

In [72]:
pred = np.exp(model.predict(test_conv_input_))
pred

array([0.62919014, 1.33416959, 0.98843697, 0.21133612, 0.98843697,
       0.76754373, 0.16405317, 0.76754373, 0.7081726 , 0.14371268,
       0.7081726 , 0.14371268, 0.7081726 , 0.70571706, 0.14076792,
       0.70571706, 0.14076792, 0.70571706, 1.19749227, 1.19749227,
       0.04802669, 1.46454476, 0.11756545])

In [73]:
pd.DataFrame([pred,test_conv['time (sec)']])

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,0.629190,1.334170,0.988437,0.211336,0.988437,0.767544,0.164053,0.767544,0.708173,0.143713,...,0.705717,0.140768,0.705717,0.140768,0.705717,1.197492,1.197492,0.048027,1.464545,0.117565
1,0.767809,1.464385,1.077104,0.191137,1.068897,0.880570,0.132695,0.881454,0.817290,0.108569,...,0.925359,0.112048,0.925256,0.111904,0.925313,1.859916,1.845435,0.040141,2.294678,0.093304


In [74]:
test_conv['predicted'] = pd.DataFrame(pred)

In [75]:
test_conv['diff'] = np.abs(test_conv.predicted-test_conv['time (sec)'])

In [83]:
test_conv['diff'].mean()

0.1740820967703774

In [84]:
test_conv.to_csv('predicted_model.csv')

In [36]:
"""
model = GBT(n_estimators=1000,max_depth=3)
model.fit(X_train, y_train)
print('accuracy: ',model.score(X_test, y_test)*100)
#pred = model.predict(X_test)
#print 'nonzero: ',pred.shape[0]-pred[pred<0].shape[0],'/',pred.shape[0]
"""

"\nmodel = GBT(n_estimators=1000,max_depth=3)\nmodel.fit(X_train, y_train)\nprint('accuracy: ',model.score(X_test, y_test)*100)\n#pred = model.predict(X_test)\n#print 'nonzero: ',pred.shape[0]-pred[pred<0].shape[0],'/',pred.shape[0]\n"